# Step 0: Installing requirements

The [conda](https://docs.conda.io/en/latest/) package manager is used to install the packages used in this workflow. If GROMACS is not already installed on your machine, it can be obtained by running `conda install -c bioconda gromacs`

In [ ]:
!conda install --file requirements.txt --yes

# Step 1: Using the TraPPE force field

The Transferable Potentials for Phase Equilibria (TraPPE) force field is commonly used for studying phase equilibria. More information can be found at these references:
1. M.G. Martin, and J.I. Siepmann, Transferable potentials for phase equilibria. 1. United-atom description of n-alkanes J. Phys. Chem. B 102, 2569-2577 (1998).

2.  M.S. Shah, M. Tsapatsis, and J.I. Siepmann, Transferable potentials for phase equilibria. Improved united-atom description of ethane and ethylene AIChE J. 63, 5098-5110 (2017).

A small portion of TraPPE has been implemented in files provided in this workflow.

# Step 2: Building the mBuild compound
To implement non-atomistic compounds and the bond-less PDB files from the TraPPE website, some modifications will be necessary after creating our `mb.Compound`

For extensibility purposes, these functions could be easily wrapped into an mBuild class.

In [ ]:
import mbuild as mb

cmpd = mb.load("TraPPE_UA_3_propane_monomer1.pdb")
cmpd.name = "Pro"

The PDB has some awkward naming conventions that manifest in the mBuild particles.
To remedy this, we will convert the particles' names to follow the non-atomistic convention, 
where particle names are prefaced with an underscore.

Furthermore, the PDB file does not contain bond information, so we will manually specify bonds in mBuild.

In [ ]:
particles = [a for a in cmpd.particles()]
particles

In [ ]:
particles[0].name = "_" + particles[0].name[1:]
particles[1].name = "_" + particles[1].name
particles[2].name = "_" + particles[2].name[1:]
cmpd.add_bond((particles[0], particles[1]))
cmpd.add_bond((particles[1], particles[2]))
particles

After "cleaning up" the mBuild compound, we can fill a box to generate a molecular system

In [ ]:
box = mb.fill_box(cmpd, n_compounds=100, box=[3, 3, 3])

# Step 3: Applying the foyer force field
The foyer XML provided on the TraPPE website requires some modification to fulfill the foyer XML schema.

Namely, the `ForceField` XML element should not have the attribute `model`. While this is useful information to document, this is not consistent with the current foyer XML schema, so the attribute needs to be removed.

Furthermore, the `HarmonicBondForce`, `HarmonicAngleForce`, and `RBTorsionForce` elements may include DOI information. Again, while this is useful information to document, this is not consistent with the current foyer XML schema, so the DOIs in these XML elements need to be removed. The DOIs in the `AtomTypes` section, however, can be kept.

The XML provided in this repo has already modified the TraPPE XML.

Note: The "canonical" TraPPE implementation utilizes *constrained bonds or fixed bond lengths*, but the listed bond constants within the XML are based on GAFF parametrization for flexible bonds. Please observe this feature within TraPPE to ensure simulation consistency. Additionally, observe that the TraPPE force field excludes 1-4 LJ and Coulombic interactions

In [ ]:
import foyer

ff = foyer.Forcefield(forcefield_files="TraPPE_UA_3_fully_flexible_propane.xml")

In [ ]:
structure = box.to_parmed(residues="Pro")
parametrized = ff.apply(structure)

After parametrizing our compound, we can save the structure to gromacs-suitable files.

In [ ]:
parametrized.save("out.gro", overwrite=True)
parametrized.save("out.top", overwrite=True)

# Step 4: Running the simulation

As with most simulation engines, an input or control file is necessary. 
For gromacs, this is the MDP file. 

* As mentioned earlier in Step 3, TraPPE typically utilizes fixed bond lengths, so we make sure to specify `constraints = all-bonds`
* As specified on the TraPPE website, cutoffs are 1.4 nm, tail corrections for LJ interactions, and Ewald summations for Coulombic interactions


In [ ]:
def write_gmx_mdp(filename):
    with open(filename, "w") as f:
        f.write(
            """
title                     = NVT Equilibration
; Run parameters
integrator                = md        ; leap-frog integrator
nsteps                    = 100000     ; 2 * 5000 = 10ps
dt                        = 0.002     ; 2 fs

; Output control
nstxout                   = 500       ; Every 1.0 ps
nstvout                   = 500
nstenergy                 = 500
nstlog                    = 500

;Bond parameters
continuation              = no
constraint_algorithm    = lincs
constraints             =   all-bonds
lincs_iter              = 1
lincs_order             = 4

; Neighbor searching
cutoff-scheme           = Verlet
nstype                  = grid
nstlist                 = 10
rcoulomb                = 1.4
rvdw                    = 1.4

; Electrostatics
coulombtype             = PME
pme_order               = 4
fourierspacing          = 0.16

; Temperature coupling
tcoupl                  = nose-hoover
tc-grps                 = system
tau_t                   = 0.4  
ref_t                   = 300  

; Pressure coupling
pcoupl                 = no

;Periodic boundary conditions
pbc                     = xyz

;Dispersion correction
DispCorr                = EnerPres

;Velocity generation
gen_vel                 = yes
gen_temp                = 300
gen_seed                = -1"""
        )

In [ ]:
import subprocess

write_gmx_mdp("nvt.mdp")
grompp_cmd = "gmx grompp -f nvt.mdp -c out.gro -p out.top -maxwarn 1 -o nvt"
p = subprocess.Popen(
    grompp_cmd,
    shell=True,
    universal_newlines=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
out, err = p.communicate()
with open("grompp.out", "w") as f:
    f.write(out)
with open("grompp.err", "w") as f:
    f.write(err)

mdrun_cmd = "gmx mdrun -deffnm nvt"
p = subprocess.Popen(
    mdrun_cmd,
    shell=True,
    universal_newlines=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
out, err = p.communicate()
with open("mdrun.out", "w") as f:
    f.write(out)
with open("mdrun.err", "w") as f:
    f.write(err)

# Step 5: Analysis

Without running enough simulations to fully equilibrate and sample a system, we can simply do a check on the energy of the system as a function of time. We use the `panedr` package to parse the EDR file, which is a binary format, to a `pandas` dataframe.

In [ ]:
import panedr

df = panedr.edr_to_df("nvt.edr")

energies = ["Potential", "LJ (SR)", "Angle", "Coulombic"]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition, inset_axes

fig, ax = plt.subplots()

axins = plt.axes([0, 0, 1, 1])
ip = InsetPosition(ax, [0.2, 0.5, 0.4, 0.4])
axins.set_axes_locator(ip)

for energy in energies:
    if energy == "Coulombic":
        ax.plot(df["Time"], df["Coulomb (SR)"] + df["Coul. recip."], label="Coulomb")
        axins.plot(df["Time"], df["Coulomb (SR)"] + df["Coul. recip."], label="Coulomb")
        continue
    else:
        ax.plot(df["Time"], df[energy], label=energy)
        axins.plot(df["Time"], df[energy], label=energy)

    ax.set_xlabel("Trajectory time, ps")
    ax.set_ylabel("System energy (kJ/mol)")

ax.set_ylim((-1000, 2000))
axins.set_xlim((-1, 20))
axins.set_ylim((-400, 4000))
ax.legend()

# Concluding Remarks
Some low-level routines for constructing a TraPPE molecular simulation have been provided. There is clear room for extensibility to build multiple, different mBuild compounds; to simulate different thermodynamic statepoints; and to do so in a very modular, reproducible fashion. Future work can include disseminating the entire TraPPE force field within an XML, validating dihedral fits within the TraPPE implementation, and adjusting the PDB/XML files on the TraPPE website to ensure MoSDeF consistency.